In [1]:
import pandas as pd

In [2]:
%%time
folder_path = 'data/'
train = pd.read_pickle(f'{folder_path}train.pkl')

CPU times: user 1.05 s, sys: 2.05 s, total: 3.1 s
Wall time: 3.12 s


In [3]:
test = pd.read_pickle(f'{folder_path}test.pkl')

In [5]:
import datetime

In [6]:
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
train['TransactionDT'] = train['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
test['TransactionDT'] = test['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))

In [7]:

def make_subcard(df):
    # account1 reg date magic feature
    df['card_registered_delta_tmp'] = pd.to_timedelta(df['D1'], unit='day')
    df['subcard_reg_date'] = (
            train['TransactionDT'] - df['card_registered_delta_tmp']
    )
    df['subcard_reg_timestamp'] = df['subcard_reg_date']\
        .dt\
        .date\
        .apply(
        lambda x: (
                x - datetime.date(1970, 1, 1)
        ).total_seconds()
    )

    df['subcard'] = df['card1'].astype(str) + '_' + df['subcard_reg_date'].dt \
        .date \
        .astype(str)# + '_' + df['ProductCD']

In [8]:
make_subcard(train)
make_subcard(test)

In [25]:
def count_full_fraud_subcard(group):
    fraud = train.groupby(['subcard'] + list(group))['isFraud'].agg(['mean', 'count'])
    return (fraud[fraud['mean'] == 1]['count']).sum()

In [26]:
categorical =  set(['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19',
            'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37',
            'id_38', 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
            'R_emaildomain', 'card2', 'card3',  'card5', 'addr1', 'addr2', 
            'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9'])

In [53]:
train['id_31'].value_counts()

chrome 63.0              22000
mobile safari 11.0       13423
mobile safari generic    11474
ie 11.0 for desktop       9030
safari generic            8195
                         ...  
chrome 69.0                  1
Nokia/Lumia                  1
Cherry                       1
Samsung/SCH                  1
firefox mobile 61.0          1
Name: id_31, Length: 130, dtype: int64

In [52]:
['addr1', 'DeviceInfo', 'P_emaildomain', 'ProductCD', 'id_20', 'id_19', 'id_31', 'card4']

['addr1',
 'DeviceInfo',
 'P_emaildomain',
 'ProductCD',
 'id_20',
 'id_19',
 'card4']

In [31]:
mean_tops = {}
for group in combinations(categorical, 1):
    cnt = count_full_fraud_subcard(group)
    print(group, cnt)
    mean_tops[group] = cnt

('id_33',) 3080
('DeviceType',) 5904
('id_34',) 2523
('id_30',) 3116
('M4',) 7650
('id_22',) 405
('id_28',) 5936
('ProductCD',) 9450
('M6',) 4641
('id_16',) 5709
('M1',) 3015
('P_emaildomain',) 10359
('id_23',) 413
('id_12',) 5037
('id_24',) 392
('id_31',) 8428
('id_35',) 5104
('M3',) 3468
('id_27',) 405
('id_19',) 8537
('id_32',) 2557
('M9',) 2732
('id_29',) 5938
('card2',) 7807
('card3',) 7782
('card6',) 7784
('id_21',) 415
('id_26',) 417
('id_14',) 2743
('card5',) 7779
('M2',) 3352
('M7',) 2590
('id_18',) 2824
('id_15',) 6315
('addr1',) 9420
('M5',) 4534
('R_emaildomain',) 6713
('addr2',) 5582
('id_20',) 8270
('card4',) 7779
('id_25',) 406
('DeviceInfo',) 7055
('id_13',) 6232
('M8',) 2721
('id_36',) 4927
('id_37',) 5458
('id_17',) 5031
('id_38',) 5224


In [ ]:
def part_of_day(hour):
    if 0 <= hour <6:
        return 'night'
    elif 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'day'
    else:
        return 'evening'

In [ ]:
search_df[search_df['subcard'] == '1078_2017-12-18']

D1: timedelta (days, rounded down) since first transaction for one card. You can see accounts with 2 cards (husband and wife using one card each from a joint account?) where there are two D1 = 0 within a few days of each other, and subsequently D1 provides a timedelta from one or other of those D = 0 transactions depending on which of the two cards was used. NaN sometimes (but rarely) appears in the D1 column, and it's not clear what that means, but I expect it means it was not the first transaction from that card, but they had no information on the date of the first one.

D1 = 0 for most isFraud=1 transactions. Presumably this is due to a new card getting intercepted in the post, or a fraudster managing to request a card i somebody else's name.

D2: this appears to be the same as D1, except D1 = 0 values have been replaced by NaN.

D3: timedelta since the previous transaction for one card. As with D1 and D2, the this feature appears to count different cards separately.

D4: timedelta since first transaction for all cards on the account. Using the example of a husband and wife each using their own card on a joint credit card account, this feature would not distinguish between which card was used.

D5: timedelta since the previous transaction for all cards on the account.

D6 and D7: some variant of D4 and D5, but the algorithm to produce them is still not clear (to me!). In some cases D6 = D4 and D5 = D7, but in other cases D6 and D7 are simply NaN. In any case, dropping features D6 and D7 reduces the ROC AUC score (using XGBoost) so there is clearly some useful additional information herein.

D8: timedelta (float) since some event. In some simple cases, the integer part of D8 is equal to D4, in other cases it is larger than D4 (or NaN). My best guess is this is the timedelta since the card was issued or activated. Obvious 'his and hers' joint credit card accounts have different D8 values for their respective cards, so the time reference cannot be when the account was opened. You can also find joint credit card accounts where, after a long time of inactivity, both cards resurface with D1=0 on the same day, and with a the time reference used in D8 now later than that used in the previous transactions.

D9 is simply D8 minus its integer part. The presence of this feature gives us a clue about what D8 means. My best guess is that D9 indicates the difference between the time of day when the card was issue/activated (?) and when it was used in the transaction. I.e., if the card was activated at 9 AM, and a transaction is then made at 9 PM, you will get D9 = 0.5 since the time of day is shifted by half a day. This feature might be useful to detect fraud committed when the cardholder and fraudster are in very different time-zones.

D10: to be completed later (I don't know). Values: 0, 1 or NaN

D11: to be completed later (I don't know). Some kind of timedelta. I don't find this feature useful for my XGBoost models. Removing the D11 feature gives me a slight increase (+0.004) in my CV and LB scores.

D12-D15: to be completed later (I don't know). D12, D13, D15 appear to be variants of D4.